## Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic
для источников paid и email из России - ставим ad
для источников paid и email не из России - ставим other
все остальные варианты берем из traffic_source без изменений

In [1]:
import pandas as pd
import numpy as np

In [2]:
def def_traffic_type(row):
    if row.region == 'Russia':
        if row.traffic_source in ['yandex', 'google']:
            return 'organic'
        elif row.traffic_source in ['paid', 'email']:
            return 'ad'
    elif row.traffic_source in ['paid', 'email']:
        return 'other'
    return row.traffic_source
    

In [3]:
log = pd.read_csv('visit_log.csv', sep=';')

In [4]:
log['traffic_type'] = log.apply(def_traffic_type, axis=1)

In [5]:
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [6]:
log.traffic_type.unique()

array(['organic', 'direct', 'ad', 'google', 'other', 'yandex'],
      dtype=object)

## Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

Прочитайте содержимое файла с датафрейм
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [8]:
urls = pd.read_csv('URLs.txt')
urls[urls.url.str.contains(r'/\d{8}-')]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


## Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [9]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.tail()

,userId,movieId,rating,timestamp
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663
100003,671,6565,3.5,1074784724


In [10]:
def users_life_time(rows):
    return rows.max() - rows.min()

def new_sum(rows):
    if rows.sum() >= 100:
        return rows.sum()
    
ratings_100 = (
ratings.groupby('userId').
    agg({'rating' : new_sum, 'timestamp' : users_life_time}).
    dropna().
    rename(columns={'rating' : 'number_of_grades', 'timestamp' : 'user_lifetime'}).
    reset_index()
)

ratings_100.head()

,userId,number_of_grades,user_lifetime
0,2,265.0,851
1,3,182.0,71198
2,4,887.0,203560
3,5,391.0,2101
4,6,143.5,1124022


In [11]:
ratings_100.user_lifetime.mean() / (60 * 60 * 24)

236.23018923406622

## Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [12]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [13]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [14]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [15]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [18]:
for n in range(5):
    print(n)

0
1
2
3
4


In [37]:
def crazy_merge(list_of_df, how, on):
    while len(list_of_df) > 2:
        df = list_of_df.pop().merge(list_of_df.pop(), how=how, on=on)
        list_of_df.append(df)
    return list_of_df[-1].merge(list_of_df[0], how=how, on=on)

crazy_merge([rzd, auto, air, client_base], how='outer', on='client_id')

,client_id,address,air_revenue,auto_revenue,rzd_revenue
0,111,Комсомольская 4,NaN,NaN,1093.0
1,112,Энтузиастов 8а,NaN,NaN,2810.0
2,113,Левобережная 1а,NaN,57483.0,10283.0
3,114,Мира 14,NaN,83.0,5774.0
4,115,ЗЖБИиДК 1,81.0,912.0,981.0
5,116,Строителей 18,4.0,4834.0,NaN
6,117,Панфиловская 33,13.0,98.0,NaN
7,118,Мастеркова 4,173.0,NaN,NaN


In [38]:
crazy_merge([rzd, auto, air], how='outer', on='client_id')

,client_id,air_revenue,auto_revenue,rzd_revenue
0,115,81.0,912.0,981.0
1,116,4.0,4834.0,NaN
2,117,13.0,98.0,NaN
3,118,173.0,NaN,NaN
4,113,NaN,57483.0,10283.0
5,114,NaN,83.0,5774.0
6,111,NaN,NaN,1093.0
7,112,NaN,NaN,2810.0


### Домашнее задание
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [53]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})
data

,keyword,shows
0,курс гривны к рублю,125076
1,доллары в рубли,114173
2,100 долларов в рублях,97534
3,курс рубля,53546


In [54]:
from pymystem3 import Mystem

def lemmatize(row):
    m = Mystem()
    return ' '.join(m.lemmatize(row)).replace('   ', ' ')[:-2]

In [55]:
data['lemmas'] = data.keyword.apply(lemmatize)
data

,keyword,shows,lemmas
0,курс гривны к рублю,125076,курс гривна к рубль
1,доллары в рубли,114173,доллар в рубль
2,100 долларов в рублях,97534,100 доллар в рубль
3,курс рубля,53546,курс рубль
